- ¿Cuáles fueron los géneros mejor rankeados en promedio en el año 2005?
- ¿Cuál es el top 5 de películas mejor rankeadas en promedio?
- ¿Cuál es el género con más revies?
- ¿Cuáles es el género por año con mejor rating entre el año 2000 y 2010?
- ¿Cuál es el usuario con más críticas y cuál es su género favorito?

In [1]:
import pandas as pd
import numpy as np
import itertools

In [2]:
ratings = pd.read_csv('../data/ratings.csv')
print(ratings.shape)
ratings.head()

(100836, 4)


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [3]:
movies = pd.read_csv('../data/movies.csv')
print(movies.shape)
movies.head()

(9742, 3)


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


## ¿Cuáles fueron los géneros mejor rankeados en promedio en el año 2005?

In [4]:
movies_groupby = ratings.groupby(['movieId'])

rated_movies = movies_groupby.mean().merge(movies, on='movieId').drop(columns=['userId', 'timestamp'])
# rated_movies.sort_values('rating', ascending=False).head()

In [5]:
rated_movies['genres'] = rated_movies['genres'].str.split('|')
# rated_movies.head()

In [6]:
# from https://stackoverflow.com/questions/40569402/pandas-create-several-rows-from-column-that-is-a-list
def melt_series(s):
    lengths = s.str.len().values
    flat = [i for i in itertools.chain.from_iterable(s.values.tolist())]
    idx = np.repeat(s.index.values, lengths)
    return pd.Series(flat, idx, name=s.name)
    
movies_2005 = rated_movies[rated_movies.title.str.endswith('(2005)')]
movies_2005 = movies_2005.drop(columns=['genres']).join(melt_series(movies_2005.genres).to_frame()).reset_index(drop=True)
movies_2005.head()

,movieId,rating,title,genres
0,27618,2.75,"Sound of Thunder, A (2005)",Action
1,27618,2.75,"Sound of Thunder, A (2005)",Adventure
2,27618,2.75,"Sound of Thunder, A (2005)",Drama
3,27618,2.75,"Sound of Thunder, A (2005)",Sci-Fi
4,27618,2.75,"Sound of Thunder, A (2005)",Thriller


In [7]:
best_genres = movies_2005.drop(columns=['movieId', 'title']).groupby('genres').mean().sort_values(by='rating', ascending=False)
best_genres.head()

,rating
genres,
Film-Noir,4.077381
War,3.781869
Documentary,3.773495
Animation,3.443075
IMAX,3.426322


## ¿Cuál es el top 5 de películas mejor rankeadas en promedio?

In [19]:
rated_movies.sort_values(by='rating', ascending=False).head()

,movieId,rating,title,genres
7638,88448,5.0,Paper Birds (Pájaros de papel) (2010),"[Comedy, Drama]"
8089,100556,5.0,"Act of Killing, The (2012)",[Documentary]
9065,143031,5.0,Jump In! (2007),"[Comedy, Drama, Romance]"
9076,143511,5.0,Human (2015),[Documentary]
9078,143559,5.0,L.A. Slasher (2015),"[Comedy, Crime, Fantasy]"


In [21]:
# Películas con promedio de rating == 5.0
len(rated_movies.loc[rated_movies.rating == 5])

296

## ¿Cuál es el género con más reviews?

In [39]:
ratings_by_genre = rated_movies.drop(columns=['genres']).join(melt_series(rated_movies.genres).to_frame()).reset_index(drop=True)
ratings_by_genre = ratings_by_genre.drop(columns=['movieId', 'title']).groupby('genres').count()
ratings_by_genre.sort_values(by='rating', ascending=False).head(3)

,rating
genres,
Drama,4349
Comedy,3753
Thriller,1889


In [27]:
movies_groupby.count()['userId']
# movies['ratings_mean'] = movies_groupby.mean()['rating']

movieId
1         215
2         110
3          52
4           7
5          49
         ... 
193581      1
193583      1
193585      1
193587      1
193609      1
Name: userId, Length: 9724, dtype: int64

In [39]:
ratings_per_user = ratings.groupby(['userId']).count()
ratings_per_user['rating'].sort_values(ascending=False)[:5]

userId
414    2698
599    2478
474    2108
448    1864
274    1346
Name: rating, dtype: int64